<pre>
Copyright 2021 Boris Shminke

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
</pre>

In [1]:
# since both Jupyter and `isabelle-client` use `asyncio` we need to enable
# nested event loops. We don't need that when using `isabelle-client` from
# Python scripts outside Jupyter
import nest_asyncio

nest_asyncio.apply()

In [2]:
from isabelle_client import start_isabelle_server
# first, we start Isabelle server
server_info, _ = start_isabelle_server(
    name="test", port=9999, log_file="server.log"
)

In [3]:
import logging

from isabelle_client import get_isabelle_client
# then we create Python client to Isabelle server
isabelle = get_isabelle_client(server_info)
!rm -f session.log
# we will log all the messages from the server to a file
isabelle.logger = logging.getLogger()
isabelle.logger.setLevel(logging.INFO)
isabelle.logger.addHandler(logging.FileHandler("session.log"))

In [4]:
# suppose we have an Isabelle theory file
# (probably generated by another Python script)
!cat Example.thy

theory Example
imports Main
begin
lemma "\<forall> x. \<exists> y. x = y"
by auto
end

In [5]:
# we can send this file to the server and get a response
isabelle.use_theories(
    theories=["Example"], master_dir=".", watchdog_timeout=0
)

IsabelleResponse(response_type='FINISHED', response_body='{"ok":true,"errors":[],"nodes":[{"messages":[],"exports":[],"status":{"percentage":100,"unprocessed":0,"running":0,"finished":7,"failed":0,"total":7,"consolidated":true,"canceled":false,"ok":true,"warned":0},"theory_name":"Draft.Example","node_name":"Example.thy"}],"task":"dee37ec4-d737-4d15-9f1a-3ddce2533ec6"}', response_length=322)

In [6]:
# if we have a session description using ROOT file like this
!rm -rf output
!cat ROOT

session examples = HOL +
  options [document = pdf, document_output = "output"]
  theories
    Example
  document_files
    "root.tex"


In [7]:
# we can build session (`isabelle build`)
isabelle.session_build(dirs=["."], session="examples")

IsabelleResponse(response_type='FAILED', response_body='{"task":"f48fa7d2-305c-4a63-882d-0fc55ce8f172","sessions":[{"session":"Pure","ok":true,"timeout":false,"timing":{"elapsed":0,"cpu":0,"gc":0},"return_code":0},{"session":"HOL","ok":true,"timeout":false,"timing":{"elapsed":0,"cpu":0,"gc":0},"return_code":0},{"session":"examples","ok":false,"timeout":false,"timing":{"elapsed":5.569,"cpu":8.404,"gc":0},"return_code":1}],"return_code":1,"message":"Session build failed: return code 1 (ERROR)","ok":false,"kind":"error"}', response_length=475)

In [8]:
# the results will appear in `output` folder
# notice that `output` folder does not contain a PDF file
# because `build` command failed
# (this container has no LaTeX installed)
!ls output

document


In [9]:
from isabelle_client import async_run
# or we can issue a free-text command through TCP
# here `asynchronous` argument means 'asynchronous command' as defined
# in section 4.2.6. of Isabelle System manual. `echo` is a synchronous command
# but Python communicates with Isabelle asynchronously as always
async_run(isabelle.execute_command("echo 42", asynchronous=False))

IsabelleResponse(response_type='OK', response_body='42', response_length=None)

In [10]:
# we can also shut down the Isabelle server
isabelle.shutdown()

IsabelleResponse(response_type='OK', response_body='', response_length=None)

In [11]:
# all our communications with the server
# were written to the log file specified earlier
!cat session.log

a37c3105-1cc4-41c1-a527-8da49bdadfe4
session_start {"session": "HOL"}

OK {"isabelle_id":"7e2a9a8c2b85","isabelle_version":"Isabelle2021: February 2021"}
OK {"task":"4dfe9c5c-c970-4313-af3f-75476df7d4c7"}
107
NOTE {"kind":"writeln","message":"Starting session HOL ...","task":"4dfe9c5c-c970-4313-af3f-75476df7d4c7"}
170
FINISHED {"session_id":"28133236-8b8d-4894-a522-01c4a91ff391","tmp_dir":"/tmp/isabelle-/server_session7856490314677295245","task":"4dfe9c5c-c970-4313-af3f-75476df7d4c7"}
a37c3105-1cc4-41c1-a527-8da49bdadfe4
use_theories {"session_id": "28133236-8b8d-4894-a522-01c4a91ff391", "theories": ["Example"], "watchdog_timeout": 0, "master_dir": "."}

OK {"isabelle_id":"7e2a9a8c2b85","isabelle_version":"Isabelle2021: February 2021"}
OK {"task":"dee37ec4-d737-4d15-9f1a-3ddce2533ec6"}
161
NOTE {"percentage":99,"task":"dee37ec4-d737-4d15-9f1a-3ddce2533ec6","message":"theory Draft.Example 99%","kind":"writeln","session":"","theory":"Draft.Example"}
163
NOTE {"percentage":100,"task":"dee